In [13]:
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.utils import data
from torch import optim 
import numpy as np
import matplotlib.pyplot as plt
import gc
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [8]:
sensor = np.loadtxt('../data/processed_data/sensor_data.txt')
activity = np.loadtxt('../data/processed_data/activity_data.txt')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(sensor, activity, test_size=0.3, shuffle=False)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56007, 14), (24003, 14), (56007, 16), (24003, 16))

In [10]:
sensor_dataset = data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
sensor_dataloader = data.DataLoader(sensor_dataset, shuffle=False, batch_size=56007)

## Try a encoder-decoder where encoder is a 1D convnet and decoder is a GRU

In [33]:
class EncoderCNN(nn.Module):
    def __init__(self, input_height):
        super(EncoderCNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, kernel_size=(input_height, 3), stride=1, padding=0),
            nn.MaxPool1d(kernel_size=(3, ), stride=1, padding=0),
            nn.LeakyReLU()
        self.conv2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=(3, ), stride=1, padding=0),
            nn.MaxPool1d(kernel_size=(3, ), stride=1, padding=0),
            nn.LeakyReLU()
        )

    def forward(self, input):
        output = self.conv1(input)
        output = self.conv2(output)
        return output
    
    def init_weights(self):
        for layer in self.modules():
            if isinstance(layer, nn.Conv1d):
                nn.init.xavier_normal(layer.weight)
    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(output_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input, hidden):
#         output = self.embedding(input).view(1, 1, -1)
        output = F.relu(input.view(1, 1, -1))
#         print(output.size(), hidden.size())
        output, hidden = self.gru(output, hidden)
        output = self.sigmoid(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1, 1, self.hidden_size))
    
## 0.5以下为0，以上为1
def cut_by_half(tensor):
    l = tensor.tolist()
    return Variable(torch.FloatTensor(np.asarray([1. if i >= 0.5 else 0 for i in l])))

In [40]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion, teacher_forcing_ratio=0.5):

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    assert(input_length == target_length)

    loss = 0

    encoder_output = encoder(input_tensor)
    print(encoder_output.size())
    
    decoder_input = target_tensor[0]

    decoder_hidden = encoder_output

    use_teacher_forcing = True if np.random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_input = target_tensor[di]  # Teacher forcing
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output.squeeze(), target_tensor[di])
            
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            decoder_input = cut_by_half(decoder_output.data.squeeze())
#             print(decoder_input)
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topv.squeeze().detach().type(torch.FloatTensor)  # detach from history as input
            loss += criterion(decoder_output.squeeze(), target_tensor[di])

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0]

In [41]:
def trainIters(dataloader, encoder, decoder, n_iters, learning_rate=0.01):
    encoder.init_weights()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
#     training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.MSELoss()
    print('Begin Training')
    for iter in range(1, n_iters + 1):
#         training_pair = training_pairs[iter - 1]
        ## 这里的decoder的猜测还没解决
        print("Training [{}/{}] epoch".format(iter, n_iters))
        for i, (X_batch, y_batch) in enumerate(dataloader):
            loss = train(Variable(X_batch), Variable(y_batch), encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
#             print_loss_total += loss
#             plot_loss_total += loss
           
            plot_losses.append(loss)
    return plot_losses

In [42]:
encoder = EncoderCNN()
decoder = DecoderRNN(10, activity.shape[1])

losses = trainIters(sensor_dataloader, encoder, decoder, 3, learning_rate=0.001)

D:\Anaconda3_501\lib\site-packages\ipykernel_launcher.py:24: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.


Begin Training
Training [1/3] epoch


RuntimeError: Expected 3-dimensional input for 3-dimensional weight [16, 1, 3], but got input of size [56007, 14] instead

In [32]:
for k in encoder.modules():
    print(k)
    print('----------------------')

EncoderCNN(
  (conv1): Sequential(
    (0): Conv1d(1, 16, kernel_size=(3,), stride=(1,))
    (1): MaxPool1d(kernel_size=(3,), stride=1, padding=0, dilation=1, ceil_mode=False)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (conv2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (1): MaxPool1d(kernel_size=(3,), stride=1, padding=0, dilation=1, ceil_mode=False)
    (2): LeakyReLU(negative_slope=0.01)
  )
)
----------------------
Sequential(
  (0): Conv1d(1, 16, kernel_size=(3,), stride=(1,))
  (1): MaxPool1d(kernel_size=(3,), stride=1, padding=0, dilation=1, ceil_mode=False)
  (2): LeakyReLU(negative_slope=0.01)
)
----------------------
Conv1d(1, 16, kernel_size=(3,), stride=(1,))
----------------------
MaxPool1d(kernel_size=(3,), stride=1, padding=0, dilation=1, ceil_mode=False)
----------------------
LeakyReLU(negative_slope=0.01)
----------------------
Sequential(
  (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  (1): MaxPool1d(kernel_size=(3,), stride=1, p